In [69]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
from sklearn.preprocessing import MinMaxScaler

from tensorflow.keras import Sequential
from tensorflow.keras.layers import Dense, LSTM, Dropout

ModuleNotFoundError: No module named 'tensorflow'

In [48]:
data = pd.read_csv('../Crypto_data/BITCOIN_Data.csv', date_parser = True)
data.tail()

,Datum,Zuletzt,Eröffn.,Hoch,Tief,Vol.,+/- %
703,05.01.2020,"7.372,5","7.376,8","7.501,0","7.345,6","628,14K","-0,06%"
704,04.01.2020,"7.376,8","7.345,1","7.433,1","7.291,4","523,91K","0,46%"
705,03.01.2020,"7.343,1","6.967,1","7.402,9","6.884,1","936,29K","5,40%"
706,02.01.2020,"6.967,0","7.199,7","7.209,6","6.901,4","632,78K","-3,23%"
707,01.01.2020,"7.199,8","7.196,4","7.259,4","7.180,0","420,28K","0,05%"


In [54]:
data.rename(columns={'Datum':'Date', 'Eröffn.':'Open', 'Hoch':'High',
                     'Tief':'Low', 'Vol.':'Volume', 'Zuletzt':'Adj Close'}, inplace=True)
data.drop('+/- %', axis=1, inplace=True)

In [55]:
for column in data.drop(columns=['Date', 'Volume']).columns:
    data[column] = [float(str(i).replace(",", "")) for i in data[column]]
    
data['Volume'] = [str(i).replace(",", "") for i in data['Volume']]

In [56]:
data

,Date,Adj Close,Open,High,Low,Volume
0,08.12.2021,50.4971,50.5966,50.7991,50.0909,4886000.0
1,07.12.2021,50.5952,50.5474,51.9186,50.0709,5629000.0
2,06.12.2021,50.5621,49.4121,50.9133,47.2379,8968000.0
3,05.12.2021,49.4055,49.1964,49.6893,47.7978,7203000.0
4,04.12.2021,49.1952,53.6207,53.8472,42.5878,16800000.0
...,...,...,...,...,...,...
703,05.01.2020,7.3725,7.3768,7.5010,7.3456,62814000.0
704,04.01.2020,7.3768,7.3451,7.4331,7.2914,52391000.0
705,03.01.2020,7.3431,6.9671,7.4029,6.8841,93629000.0
706,02.01.2020,6.9670,7.1997,7.2096,6.9014,63278000.0


In [57]:
def value_to_float(x):
    if type(x) == float or type(x) == int:
        return x
    if 'K' in x:
        if len(x) > 1:
            return float(x.replace('K', '')) * 1000
        return 1000.0
    if 'M' in x:
        if len(x) > 1:
            return float(x.replace('M', '')) * 1000000
        return 1000000.0
    if 'B' in x:
        return float(x.replace('B', '')) * 1000000000
    return 0.0

data['Volume'] = data['Volume'].apply(value_to_float)

In [58]:
data_training = data[data['Date']< '2020-01-01'].copy()
data_test = data[data['Date']< '2020-01-01'].copy()

In [59]:
training_data = data_training.drop(['Date', 'Adj Close'], axis = 1)
training_data.head()

,Open,High,Low,Volume
0,50.5966,50.7991,50.0909,0.0
1,50.5474,51.9186,50.0709,0.0
2,49.4121,50.9133,47.2379,0.0
3,49.1964,49.6893,47.7978,0.0
4,53.6207,53.8472,42.5878,0.0


In [61]:
# MinMaxScaler is used to normalize the data
scaler = MinMaxScaler()
training_data = scaler.fit_transform(training_data)
training_data

array([[0.73000869, 0.71406589, 0.73995204, 0.        ],
       [0.72922417, 0.73166219, 0.73963186, 0.        ],
       [0.71112121, 0.71586088, 0.69427875, 0.        ],
       ...,
       [0.03431319, 0.03196414, 0.04826032, 0.        ],
       [0.03802212, 0.02892585, 0.04853727, 0.        ],
       [0.0379695 , 0.0297086 , 0.05299734, 0.        ]])

In [62]:
X_train = [] 
Y_train = []

In [63]:
training_data.shape[0]

468

In [65]:
for i in range(60, training_data.shape[0]):
    X_train.append(training_data[i-60:i])

In [66]:
Y_train.append(training_data[i,0])

In [67]:
X_train, Y_train = np.array(X_train), np.array(Y_train)
X_train.shape

(408, 60, 4)